# Web Scraping Wikipedia and Geocoding with GeoPy

This notebook aims to add location data that can replace missing values in my College Football Transfer Portal project. To source the latitude and longitude of Division 1 FCS teams, I scrape a table from Wikipedia that contains a list of all teams with the city/state for each team. I then use GeoPy to return the latitude and longitude for each team, and then save the details to a csv to be leveraged in the Transfer Portal project.

### Installing and Importing Packages

In [1]:
! pip install geopy 
! pip install Nominatim

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Web Scraping

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_FCS_football_programs'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'class': "wikitable"})

In [4]:
teams = pd.read_html(str(table))
teams = pd.DataFrame(teams[0])
pd.set_option('display.max_rows', None)
teams.head()

,Team,Name,School,City,State[1],Programestablished,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}FirstFCSseason,Conference
0,Abilene Christian,Wildcats,Abilene Christian University,Abilene,Texas,1919,2017,WAC
1,Alabama A&M,Bulldogs,Alabama A&M University,Normal,Alabama,1912,1999,SWAC
2,Alabama State,Hornets,Alabama State University,Montgomery,Alabama,1901,1982,SWAC
3,Albany,Great Danes,"University at Albany, SUNY",Albany,New York,1973,1999,CAA
4,Alcorn State,Braves,Alcorn State University,Lorman,Mississippi,1921,1978,SWAC


### Data Cleaning

In [5]:
pd.set_option('display.max_rows', None)

teams = teams.rename(columns = {"State[1]": "State"})
teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 8 columns):
 #   Column                                                                                 Non-Null Count  Dtype 
---  ------                                                                                 --------------  ----- 
 0   Team                                                                                   123 non-null    object
 1   Name                                                                                   123 non-null    object
 2   School                                                                                 123 non-null    object
 3   City                                                                                   123 non-null    object
 4   State                                                                                  123 non-null    object
 5   Programestablished                                                                    

In [6]:
teams = teams[['Team','Name','School','City','State']]
teams.head()

,Team,Name,School,City,State
0,Abilene Christian,Wildcats,Abilene Christian University,Abilene,Texas
1,Alabama A&M,Bulldogs,Alabama A&M University,Normal,Alabama
2,Alabama State,Hornets,Alabama State University,Montgomery,Alabama
3,Albany,Great Danes,"University at Albany, SUNY",Albany,New York
4,Alcorn State,Braves,Alcorn State University,Lorman,Mississippi


In [7]:
# Converting the State field to an abbreviation

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC"
}

In [8]:
teams = teams.replace({"State": us_state_to_abbrev})
teams.head()

,Team,Name,School,City,State
0,Abilene Christian,Wildcats,Abilene Christian University,Abilene,TX
1,Alabama A&M,Bulldogs,Alabama A&M University,Normal,AL
2,Alabama State,Hornets,Alabama State University,Montgomery,AL
3,Albany,Great Danes,"University at Albany, SUNY",Albany,NY
4,Alcorn State,Braves,Alcorn State University,Lorman,MS


In [9]:
# Create a Location field using the school name and country name in geocoding
# If the location field does not yield coordinates, we can use the city_state field for geocoding

teams['Location'] = teams['School'] + ", " + "United States"
teams['city_state'] = teams['City'] + ", " + teams['State']
teams.head()

,Team,Name,School,City,State,Location,city_state
0,Abilene Christian,Wildcats,Abilene Christian University,Abilene,TX,"Abilene Christian University, United States","Abilene, TX"
1,Alabama A&M,Bulldogs,Alabama A&M University,Normal,AL,"Alabama A&M University, United States","Normal, AL"
2,Alabama State,Hornets,Alabama State University,Montgomery,AL,"Alabama State University, United States","Montgomery, AL"
3,Albany,Great Danes,"University at Albany, SUNY",Albany,NY,"University at Albany, SUNY, United States","Albany, NY"
4,Alcorn State,Braves,Alcorn State University,Lorman,MS,"Alcorn State University, United States","Lorman, MS"


In [10]:
from geopy.geocoders import Nominatim
import numpy as np
from geopy.extra.rate_limiter import RateLimiter

# If the Location column does not yield a geocode, then the city_state column will be geocoded.

geocoder = RateLimiter(Nominatim(user_agent = 'RP').geocode, min_delay_seconds = 1)
teams['Address'] = np.where(teams['Location'].apply(geocoder).isnull(), teams['city_state'].apply(geocoder), teams['Location'].apply(geocoder))

# Storing the latitude/longitude attributes of the geocode result in separate columns

teams['Latitude'] = teams['Address'].apply(lambda loc: loc.latitude if loc else None)
teams['Longitude'] = teams['Address'].apply(lambda loc: loc.longitude if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('North Carolina A&T State University, United States',), **{}).
Traceback (most recent call last):
  File "C:\Users\Rahul\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\Rahul\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Rahul\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Rahul\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Rahul\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Rahul\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Rahul\anaconda3\lib\ssl.p

RateLimiter swallowed an error after 2 retries. Called with (*('North Carolina A&T State University, United States',), **{}).
Traceback (most recent call last):
  File "C:\Users\Rahul\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\Rahul\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Rahul\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Rahul\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Rahul\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Rahul\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Rahul\anaconda3\lib\ssl.py",

In [11]:
# Check to make sure there are no teams with null locations
teams[teams['Latitude'].isnull()]

,Team,Name,School,City,State,Location,city_state,Address,Latitude,Longitude


In [12]:
teams['Team'] = teams['Team'].replace({"Sam Houston": "Sam Houston State", "LIU[e]": "LIU", "McNeese State": "McNeese", "Penn": "Pennsylvania"})

In [13]:
teams = teams.add_suffix("_FCS")

In [14]:
teams = teams[['Team_FCS', 'City_FCS', 'State_FCS', 'Latitude_FCS', 'Longitude_FCS']]
teams.head()

,Team_FCS,City_FCS,State_FCS,Latitude_FCS,Longitude_FCS
0,Abilene Christian,Abilene,TX,32.469973,-99.707360
1,Alabama A&M,Normal,AL,34.783895,-86.568423
2,Alabama State,Montgomery,AL,32.363350,-86.293551
3,Albany,Albany,NY,42.651167,-73.754968
4,Alcorn State,Lorman,MS,31.876749,-91.141797


In [15]:
teams.to_csv(r'Documents\FCS Teams.csv', index = False)